In [158]:
import os

In [159]:
%pwd

'C:\\Users\\DEELL\\Desktop\\Wine-Quality'

In [160]:
#os.chdir("../")
os.chdir("C:/Users/DEELL/Desktop/Wine-Quality")

In [161]:
%pwd

'C:\\Users\\DEELL\\Desktop\\Wine-Quality'

In [162]:
from dataclasses import dataclass
from pathlib import Path

In [163]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [171]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml,create_directories

In [166]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
    
        create_directories( [self.config.artifacts_root])
        print(self.config)
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config


In [167]:
import os
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utils.common import get_size

In [168]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file 
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [172]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e



[2024-03-15 18:12:21,886: INFO: common:  yaml file : config\config.yaml loaded successfully]
[2024-03-15 18:12:21,984: INFO: common:  yaml file : params.yaml loaded successfully]
[2024-03-15 18:12:22,028: INFO: common:  yaml file : schema.yaml loaded successfully]
[2024-03-15 18:12:22,067: INFO: common: created directories at :artifacts]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/winequality-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2024-03-15 18:12:22,107: INFO: common: created directories at :artifacts/data_ingestion]
[2024-03-15 18:12:24,474: INFO: 574582996: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zi